In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points


from subprocess import check_output
from sklearn import preprocessing

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,Product_Identifier,Supermarket_Identifier,Product_Supermarket_Identifier,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type,Product_Supermarket_Sales
0,DRA12,CHUKWUDI010,DRA12_CHUKWUDI010,11.600,Low Fat,0.069,Soft Drinks,357.540,2005,NaN,Cluster 3,Grocery Store,709.080
1,DRA12,CHUKWUDI013,DRA12_CHUKWUDI013,11.600,Low Fat,0.041,Soft Drinks,355.790,1994,High,Cluster 3,Supermarket Type1,6381.690
2,DRA12,CHUKWUDI017,DRA12_CHUKWUDI017,11.600,Low Fat,0.041,Soft Drinks,350.790,2014,NaN,Cluster 2,Supermarket Type1,6381.690
3,DRA12,CHUKWUDI018,DRA12_CHUKWUDI018,11.600,Low Fat,0.041,Soft Drinks,355.040,2016,Medium,Cluster 3,Supermarket Type2,2127.230
4,DRA12,CHUKWUDI035,DRA12_CHUKWUDI035,11.600,Ultra Low fat,0.000,Soft Drinks,354.790,2011,Small,Cluster 2,Supermarket Type1,2481.770


In [4]:
train.tail()

,Product_Identifier,Supermarket_Identifier,Product_Supermarket_Identifier,Product_Weight,Product_Fat_Content,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket _Size,Supermarket_Location_Type,Supermarket_Type,Product_Supermarket_Sales
4985,NCZ54,CHUKWUDI019,NCZ54_CHUKWUDI019,nan,Low Fat,0.146,Household,402.390,1992,Small,Cluster 1,Grocery Store,406.140
4986,NCZ54,CHUKWUDI027,NCZ54_CHUKWUDI027,nan,Low Fat,0.083,Household,410.140,1992,Medium,Cluster 3,Supermarket Type3,13808.690
4987,NCZ54,CHUKWUDI045,NCZ54_CHUKWUDI045,14.650,Low Fat,0.084,Household,406.140,2009,NaN,Cluster 2,Supermarket Type1,5685.930
4988,NCZ54,CHUKWUDI046,NCZ54_CHUKWUDI046,14.650,Low Fat,0.083,Household,404.890,2004,Small,Cluster 1,Supermarket Type1,11778.000
4989,NCZ54,CHUKWUDI049,NCZ54_CHUKWUDI049,14.650,Low Fat,0.083,Household,403.390,2006,Medium,Cluster 1,Supermarket Type1,17870.070


In [5]:
train.pop('Product_Identifier')
train.pop('Supermarket_Identifier')
train.pop('Product_Fat_Content')
train.pop('Product_Weight')
train.pop('Supermarket _Size')

0          NaN
1         High
2          NaN
3       Medium
4        Small
5          NaN
6          NaN
7         High
8          NaN
9        Small
10      Medium
11       Small
12      Medium
13         NaN
14      Medium
15       Small
16      Medium
17       Small
18      Medium
19      Medium
20      Medium
21         NaN
22       Small
23      Medium
24         NaN
25       Small
26         NaN
27      Medium
28         NaN
29        High
         ...  
4960     Small
4961    Medium
4962       NaN
4963       NaN
4964     Small
4965     Small
4966       NaN
4967       NaN
4968      High
4969     Small
4970       NaN
4971      High
4972       NaN
4973    Medium
4974      High
4975       NaN
4976    Medium
4977     Small
4978       NaN
4979     Small
4980    Medium
4981    Medium
4982    Medium
4983       NaN
4984    Medium
4985     Small
4986    Medium
4987       NaN
4988     Small
4989    Medium
Name: Supermarket _Size, Length: 4990, dtype: object

In [6]:
test.pop('Product_Identifier')
test.pop('Supermarket_Identifier')
test.pop('Product_Fat_Content')
test.pop('Product_Weight')
test.pop('Supermarket _Size')

0          NaN
1         High
2         High
3          NaN
4         High
5          NaN
6       Medium
7        Small
8          NaN
9       Medium
10       Small
11       Small
12      Medium
13         NaN
14      Medium
15      Medium
16         NaN
17       Small
18       Small
19         NaN
20      Medium
21       Small
22      Medium
23         NaN
24       Small
25         NaN
26      Medium
27      Medium
28        High
29         NaN
         ...  
3502     Small
3503       NaN
3504     Small
3505    Medium
3506       NaN
3507    Medium
3508     Small
3509     Small
3510      High
3511    Medium
3512     Small
3513    Medium
3514       NaN
3515    Medium
3516    Medium
3517       NaN
3518     Small
3519    Medium
3520     Small
3521    Medium
3522       NaN
3523    Medium
3524     Small
3525       NaN
3526       NaN
3527       NaN
3528      High
3529    Medium
3530       NaN
3531      High
Name: Supermarket _Size, Length: 3532, dtype: object

In [7]:
train.head()

,Product_Supermarket_Identifier,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket_Location_Type,Supermarket_Type,Product_Supermarket_Sales
0,DRA12_CHUKWUDI010,0.069,Soft Drinks,357.540,2005,Cluster 3,Grocery Store,709.080
1,DRA12_CHUKWUDI013,0.041,Soft Drinks,355.790,1994,Cluster 3,Supermarket Type1,6381.690
2,DRA12_CHUKWUDI017,0.041,Soft Drinks,350.790,2014,Cluster 2,Supermarket Type1,6381.690
3,DRA12_CHUKWUDI018,0.041,Soft Drinks,355.040,2016,Cluster 3,Supermarket Type2,2127.230
4,DRA12_CHUKWUDI035,0.000,Soft Drinks,354.790,2011,Cluster 2,Supermarket Type1,2481.770


In [8]:
print(train.isnull().sum())

Product_Supermarket_Identifier    0
Product_Shelf_Visibility          0
Product_Type                      0
Product_Price                     0
Supermarket_Opening_Year          0
Supermarket_Location_Type         0
Supermarket_Type                  0
Product_Supermarket_Sales         0
dtype: int64


In [9]:
len(train)

4990

In [10]:
print(test.isnull().sum())

Product_Supermarket_Identifier    0
Product_Shelf_Visibility          0
Product_Type                      0
Product_Price                     0
Supermarket_Opening_Year          0
Supermarket_Location_Type         0
Supermarket_Type                  0
dtype: int64


In [11]:
len(test)

3532

In [12]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [13]:
train.head(2)

,Product_Supermarket_Identifier,Product_Shelf_Visibility,Product_Type,Product_Price,Supermarket_Opening_Year,Supermarket_Location_Type,Supermarket_Type,Product_Supermarket_Sales
0,DRA12_CHUKWUDI010,0.069,Soft Drinks,357.540,2005,Cluster 3,Grocery Store,709.080
1,DRA12_CHUKWUDI013,0.041,Soft Drinks,355.790,1994,Cluster 3,Supermarket Type1,6381.690


In [14]:
from sklearn.preprocessing import LabelEncoder
cols = ('Product_Supermarket_Identifier', 'Product_Type', 'Supermarket_Location_Type', 'Supermarket_Type')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(train[c].values)) 
    train[c] = lbl.transform(list(train[c].values))

# shape        
print('Shape all_data: {}'.format(train.shape))

Shape all_data: (4990, 8)


In [15]:
cols = ('Product_Supermarket_Identifier', 'Product_Type', 'Supermarket_Location_Type', 'Supermarket_Type')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(test[c].values)) 
    test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape all_data: {}'.format(test.shape))

Shape all_data: (3532, 7)


In [17]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [18]:
X = train[['Product_Price', 'Product_Supermarket_Identifier', 'Product_Type', 'Supermarket_Location_Type', 'Supermarket_Type']]

In [19]:
y = train['Product_Supermarket_Sales']

In [20]:
test = test[['Product_Price', 'Product_Supermarket_Identifier', 'Product_Type', 'Supermarket_Location_Type', 'Supermarket_Type']]

In [21]:
X_test,X_train,y_test,y_train = train_test_split(X,y, random_state= 42, test_size=0.3)

In [22]:
from sklearn.model_selection import cross_val_score
gbc = GradientBoostingRegressor()
scores = cross_val_score(gbc,X_train, y_train, cv=5)
scores

array([0.55800673, 0.5019606 , 0.51931959, 0.59926579, 0.55228414])

In [29]:
gbc = XGBRegressor()

In [30]:
gbc.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [31]:
print('Training score is {}'.format(gbc.score(X_train, y_train)))
print('\n')
print('Testing score is {}'.format(gbc.score(X_test, y_test)))
print('\n')
print("mse: {0:.4f}\n".format((mean_squared_error(y_train, gbc.predict(X_train)))))

Training score is 0.6651612239268758


Testing score is 0.5368664884425175


mse: 6608115.6150



In [32]:
from math import sqrt

In [33]:
training_predict = gbc.predict(X_train)

In [34]:
gbc.fit(X_train, y_train)
mse = mean_squared_error(y_train, gbc.predict(X_train))
print("MSE: %.4f" % mse)

MSE: 6608115.6150


In [35]:
rms = sqrt(mean_squared_error(y_train, gbc.predict(X_train)))

In [36]:
print("rms error is: " + str(rms))

rms error is: 2570.6255299005384


In [37]:
prediction = gbc.predict(test)

In [38]:
print(prediction)

[-142.53864 6080.896   5908.4717  ... 9703.165   1278.2546  7686.6675 ]


In [39]:
submission = pd.read_csv('SampleSubmission.csv')

In [40]:
len(submission)

3532

# Predictions

In [41]:
submission['Product_Supermarket_Sales'] =prediction
#check your submission head
submission.head()

,Product_Supermarket_Identifier,Product_Supermarket_Sales
0,DRA59_CHUKWUDI010,-142.539
1,DRA59_CHUKWUDI013,6080.896
2,DRB01_CHUKWUDI013,5908.472
3,DRB13_CHUKWUDI010,-142.539
4,DRB13_CHUKWUDI013,6080.896


In [42]:
submission.to_csv('darenew.csv', index=None)